In [1]:
import numpy as np
import scipy.constants
import math
import re

In [2]:
class Vector:
    def __init__(self, x0, x1, x2):
        self.array = np.array([x0, x1, x2])
    
    def length(self):
        return np.linalg.norm(self.array)

In [3]:
class QuantityDimension:
    def __init__(self, L=0, M=0, T=0, I=0, Θ=0, N=0, J=0):
        self.L = L
        self.M = M
        self.T = T
        self.I = I
        self.Θ = Θ
        self.J = J
 
    def getDimensions():
        return M

In [4]:
class QuantityValue:
    def __init__(self, number, reference, standardUncertainty):
        self.number = number
        self.reference = reference
        self.standardUncertainty = abs(standardUncertainty)
        self.relativeUncertainty = self.standardUncertainty / abs(self.number)
        
        self.referenceList = [re.split('\^', unit) for unit in re.split('\s+', self.reference)]
        self.dimList = filter(lambda i: (i[0] != '1' and i[0] != ''), [[dim[0], int(dim[1]) if len(dim) == 2 else 1] for dim in self.referenceList])
        self.dimDict = {}
        self.updateDims()
        
    def updateDims(self):
        for i in self.dimList:
            self.dimDict[i[0]] = (self.dimDict[i[0]] + i[1]) if i[0] in self.dimDict else i[1]
        self.reference = ''
        for key, exp in list(self.dimDict.items()):
            if exp == 0:
                del self.dimDict[key]
                continue
            self.reference += '{}{}{}'.format(
                '' if self.reference == '' else ' ',
                key if exp !=0 else '',
                '^' + str(exp) if (exp != 0 and exp != 1) else '')
                               
    def __str__(self):
        exponent = math.floor(math.log10(abs(self.number)))
        engExponent = math.floor(exponent / 3) * 3
        engMantissa = self.number / 10**engExponent
        if self.standardUncertainty != 0:
            uncertaintyExponent = math.floor(math.log10(self.standardUncertainty))
            uncertaintyMantissa = self.standardUncertainty / 10**(uncertaintyExponent-1)
            numDigits = engExponent - uncertaintyExponent + 1
            uncertaintyStr = '(' + '{:.0f}'.format(uncertaintyMantissa) + ')'
        else:
            numDigits = '4'
            uncertaintyStr = '(...)'
        if engExponent != 0:
            exponentStr = 'e' + repr(engExponent)
        else:
            exponentStr = ""
        return '{:.{}f}'.format(engMantissa, numDigits) + uncertaintyStr + exponentStr + ' ' + self.reference
    
    def __repr__(self):
        return self.__str__()
    
    def invReference(self):
        referenceStr = ''
        for key in self.dimDict:
            exp = -1 * self.dimDict[key]
            referenceStr += '{}{}{}'.format(
                '' if referenceStr == '' else ' ',
                key if exp != 0 else '',
                '^' + str(exp) if (exp != 0 and exp != 1) else ''
            )
        return referenceStr
    
    def exp(self):
        if self.dimDict == {}:
            return QuantityValue(math.exp(self.number),
                                 '',
                                 math.exp(self.number) * self.standardUncertainty)
        else:
            raise TypeError("exp function only applies to quantities of dimension 1")
            
    def sqrt(self):
        result = QuantityValue(math.sqrt(self.number),
                             self.reference,
                             math.sqrt(self.number) * 0.5 * self.relativeUncertainty)
        result.dimDict = {key: int(value / 2) for key, value in self.dimDict.items()}
        result.updateDims()
        return result
            
    def __add__(self, other):
        if isinstance(other, self.__class__) and self.dimDict == other.dimDict:
            return QuantityValue(self.number + other.number,
                                 self.reference,
                                 math.sqrt(self.standardUncertainty**2 + other.standardUncertainty**2))
        elif isinstance(other, (int, float)) and self.dimDict == {}:
            return QuantityValue(other + self.number,
                             self.reference,
                             abs(other) * self.standardUncertainty)
        else:
            raise TypeError("cannot add values with different quantity dimensions",
                            self.__str__(), self.__class__, other.__str__(), other.__class__)
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self,other):
        if isinstance(other, self.__class__) and self.dimDict == other.dimDict:
            return QuantityValue(self.number - other.number,
                                 self.reference,
                                 math.sqrt(self.standardUncertainty**2 + other.standardUncertainty**2))
        else:
            raise TypeError("cannot subtract values with different quantity dimensions")
                               
    def __mul__(self, other):
        if isinstance(other, self.__class__):
            return QuantityValue(self.number * other.number,
                                 self.reference + ' ' + other.reference,
                                 abs(self.number * other.number) 
                                 * math.sqrt(self.relativeUncertainty**2 + other.relativeUncertainty**2))
        elif isinstance(other, (int, float)):
            return QuantityValue(other * self.number,
                             self.reference,
                             abs(other) * self.standardUncertainty)
        else:
            raise TypeError("types not supported for multiplication:,", self.__str__(), self.__class__, other.__str__(), other.__class__)
    
    def __rmul__(self, other):
        return self.__mul__(other)
                        
    def __truediv__(self, other):
        if isinstance(other, self.__class__):
            return QuantityValue(self.number / other.number,
                                 self.reference + ' ' + other.invReference(),
                                 abs(self.number / other.number) 
                                 * math.sqrt(self.relativeUncertainty**2 + other.relativeUncertainty**2))
        elif isinstance(other, (int, float)):
            return QuantityValue(self.number / other,
                             self.reference,
                             abs(1 / other) * self.standardUncertainty)
    
    def __rtruediv__(self, other):
        if isinstance(other, (int, float)):
            return QuantityValue(other / self.number,
                             self.reference,
                             abs(other) * math.sqrt(self.relativeUncertainty**2))
        else:
            raise TypeError("types not supported for division")
    
    def __neg__(self):
        return QuantityValue(-1 * self.number,
                             self.reference,
                             self.standardUncertainty)

In [5]:
def getScipyConstant(key):
    return QuantityValue(scipy.constants.value(key),
                  scipy.constants.unit(key),
                  scipy.constants.precision(key) * abs(scipy.constants.value(key)))

In [33]:
pi = scipy.constants.pi
G = getScipyConstant('Newtonian constant of gravitation')
c = getScipyConstant('speed of light in vacuum')
g = getScipyConstant('standard acceleration of gravity')
mu_0 = getScipyConstant('mag. constant')

- Mass flow past skyhook, $mV_O$
- Velocity change at skyhook, $V_O\Delta\theta$
- Rate of change of momentum, $mV^2_O\Delta\theta$
- Resultant force of tension, $F_T = mV^2_O\Delta\theta$

$P = \exp(\frac{-\rho g H}{Y})$

$A_T = A_0 \exp(\frac{\rho g}{Y} \cdot \frac{RH}{R+H})$

In [7]:
def areaTop(payload, density, tensileStrength, altitude):
    return payload * g / tensileStrength * (density * g / tensileStrength * altitude / (1 + altitude / R_Earth)).exp()

In [8]:
def forceTop(payload, density, tensileStrength, altitude):
    return payload * g * (density * g / tensileStrength * altitude / (1 + altitude / R_Earth)).exp()

In [65]:
linearDensity = QuantityValue(0.1, 'kg m^-1', 0)
angle = QuantityValue(0.25, '1', 0)
altitude = QuantityValue(300e3, 'm', 5e3)
R_Earth = QuantityValue(6378137, 'm', 2) #WGS84
M_Earth = QuantityValue(5.972E24, 'kg', 0)
payload = QuantityValue(1.5E3, 'kg', 0)
tensileStrength = QuantityValue(3E9, 'kg m s^-2 m^-2', 0)
massFlowRate = 1 # kg/s
massDepositionEfficiency = .5
speed = (G * M_Earth / (R_Earth + altitude)).sqrt()
ladderDensity = QuantityValue(2e3, 'kg m^-3', 500)
hookMass = QuantityValue(5e3, 'kg', 2.5e3)
P = (-ladderDensity * g / tensileStrength * altitude).exp()
ladderMass = payload / P
ringMass = ((R_Earth + altitude) * linearDensity * 2 * math.pi)
hookLift = linearDensity * speed * speed * angle
numHooks = 2
totalMass = (ringMass + (numHooks * hookMass) + (numHooks * ladderMass))
numLaunches = totalMass / QuantityValue(63800, 'kg launches^-1', 0)
force = forceTop(payload, ladderDensity, tensileStrength, altitude)
hookLoad = force + hookMass * g * R_Earth*R_Earth/(R_Earth+altitude)/(R_Earth+altitude)
area = areaTop(payload, ladderDensity, tensileStrength, altitude)
B = QuantityValue(3.5, 'kg A^-1 s^-2', 0)
I = QuantityValue(100000, 'A', 0)

In [66]:
print('Altitude: {}\nRing speed: {}\nHook capacity: {}\nHook load: {}\nDesign payload: {}\n\nRing mass: {}\nHook mass: {}\nLadder mass: {}\nTotal mass: {}\n\nLaunches: {}'.format(
    altitude, speed, hookLift, hookLoad, payload, ringMass, hookMass, ladderMass, totalMass, numLaunches))

Altitude: 300.0(50)e3 m
Ring speed: 7.7255(29)e3 m s^-1
Hook capacity: 1.49209(79)e6 kg s^-2 m
Hook load: 140(50)e3 kg m s^-2
Design payload: 1.5000(...)e3 kg

Ring mass: 4.1960(31)e6 kg
Hook mass: 5.0(25)e3 kg
Ladder mass: 10.7(52)e3 kg
Total mass: 4.227(12)e6 kg

Launches: 66.26(19) launches


In [67]:
B

3.5000(...) kg A^-1 s^-2

In [77]:
x = QuantityValue(0.001, 'm', 0)

In [78]:
B * B * x * x / mu_0

9.7482(...) kg^2 s^-4 m^2 N^-1

In [79]:
I * I * 1000 * x * mu_0 / math.pi / x

4.0000(...)e6 N